![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

# 1. Reading in and splitting the data

Now we will import neccessary libraries `pandas` and `numpy` as well as read in the provided `bank_marketing.csv` dataset before splitting into 3 separate tables.

In [75]:
# Import libraries
import pandas as pd
import numpy as np

# Read in the a csv file as a pandas dataframe 
marketing = pd.read_csv("bank_marketing.csv")

In [76]:
# Split into the three tables
client = marketing[["client_id", "age", "job", "marital", "education", 
             "credit_default", "housing", "loan"]]
campaign = marketing[["client_id", "campaign", "month", "day", 
               "duration", "pdays", "previous", "poutcome", "y"]]
economics = marketing[["client_id", "emp_var_rate", "cons_price_idx", 
                "euribor3m", "nr_employed"]]

As a way to confirm that tables are splited successfully, I will examine first 5 rows of the `client` table.

In [77]:
# First rows of client dataframe
client.head()

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


# 2. Cleaning the data

Now that we have created 3 different tables as required, I will proceed to the data cleaning process which includes `renaming`, `cleaning`, `recreating`, `deleting` certain columns from the created tables.

In [78]:
# Rename columns
client.rename(columns={"client_id": "id"}, inplace=True)

campaign.rename(columns={"duration": "contact_duration", 
                         "previous": "previous_campaign_contacts", 
                         "y": "campaign_outcome", 
                         "campaign": "number_contacts", 
                         "poutcome": "previous_outcome"}, inplace=True)

economics.rename(columns={"euribor3m": "euribor_three_months", 
                          "nr_employed": "number_employed"}, inplace=True)

In [79]:
# Recheck all column names 
print("client's column names:", client.columns.values)
print("campaign's column names:", campaign.columns.values)
print("economics's column names:", economics.columns.values)

client's column names: ['id' 'age' 'job' 'marital' 'education' 'credit_default' 'housing' 'loan']
campaign's column names: ['client_id' 'number_contacts' 'month' 'day' 'contact_duration' 'pdays'
 'previous_campaign_contacts' 'previous_outcome' 'campaign_outcome']
economics's column names: ['client_id' 'emp_var_rate' 'cons_price_idx' 'euribor_three_months'
 'number_employed']


After renaming columns properly, **cleaning columns** process will include replacing unwanted characteristics, changing the inappropriate values, and remapping the binary values.

In [80]:
# Clean education column
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)

In [81]:
# Clean job column by removing periods
client["job"] = client["job"].str.replace(".", "")

In [82]:
# Map campaign_outcome with binary values 
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, 
                                                                 "no": 0})

In [83]:
# Convert previous_outcome to binary values
campaign["previous_outcome"] = campaign["previous_outcome"].replace("nonexistent", np.NaN)
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, 
                                                                 "failure": 0})

In [84]:
# Add campaign_id column
campaign["campaign_id"] = 1 

In [85]:
# Capitalize month and day columns
campaign["month"] = campaign["month"].str.capitalize()
# Add year column
campaign["year"] = "2022"
# Convert day to string
campaign["day"] = campaign["day"].astype(str)
# Add last_contact_date column
campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]
# Convert to datetime
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], 
                                               format="%Y-%b-%d")

After using `month`, `day`, and `year` columns to create and format the `last_contact_date` column, we can remove them to ensure the columns in each subset of the data match the table displayed in the notebook.

In [86]:
# Drop unneccessary columns
campaign.drop(columns=["month", "day", "year"], inplace=True)

# 3. Saving the data 

To prepare for the database design step, we will save the 3 tables to individual csv files without an index. 

In [87]:
# Save tables to individual csv files
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

# 4. Designing the database

As the separate csv are carefuly prepared, we will write SQL code as a string to create three database tables, `client`, `campaign`, and `economics`.

In [88]:
# Store and print database_design
client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """CREATE TABLE client
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE 
);
\copy client from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""